# An Overlappoing Generations Model of Climate-Economy Interactions

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

We want to carry out an OLG model of climate-economy interactions. 

!!! skriv noget om forfatter 

ung og gammel 

Individual preferences are described by the life-cycle utility function: 

$$u_t = log(c_{yt})+\frac{1}{1+\rho} log(c_{ot+1})$$

This utility function is logarithmic in consumption, and $\rho$ is the relative weight attached to increments oc consumption in youth and old age. 

Budget constraint: 

Young: 
$$ c_{yt} + k_{t+1} = w_t + \pi_{yt} $$
Old: 
$$ c_{ot+1} = w_{t+1} + (1+r_{t+1})k_{t+1} + \pi_{ot+1}$$

FOC: 
$$\frac{c_{ot+1}}{c_{yt}} = \frac{1+r_{t+1}}{1+\rho}$$

Comsumption possibilities of the economy are characterized by the net production function: 
$$ C_t + K_{t+1} = f_{t}(K_t, N_t, E_t, T_t) + K_t $$
Thus, 
$$ C_t = n_t c_{yt} + n_{t-1} c_{ot} $$
is the aggregated level of consumption summed over all young and old individuals. While, 
$$ K_t = n_{n-1} k_t $$
is the aggregated capital stock or the summed asset holdings of all old individuals. 
The total population or supply of labor services is 
$$ N_t = n_{t-1} + n_t $$

The residual emissions of greenhouse gases is $E_t$, and the increase in mean global temperature relative to the preindustrial norm is $T_t$. 


Further, profit maximization implies that the marginal productivity of each factor is set equal to its price so that:

$$ r_t = \frac{\partial{f_t}}{\partial{K_t}}, \space w_t = \frac{\partial{f_t}}{\partial{N_t}}, \space v_t = \frac{\partial{f_t}}{\partial{E_t}}$$

Profits are zero since the production function exhibits constant returns to scale.

We assume that future climatic conditions are determined by the time path of greenhouse gas emissions:

$$ T_t = T_t(E_0,...,E_{t-1}) $$

where $\partial{T_t}/\partial{E_{t-i}} > 0 \space (i > 0)$ so that current emissions imply increased future temperatures and hence reduced future output.

Howarth (1996) shows that any allocation that satisfies the conditions outlined above constitutes a competitive equilibrium provided that the set of lump-sum transfers ($\pi_{yt}$ and $\pi_{ot}$) and greenhouse gas emissions taxes is chosen to obey the balanced budget condition:

$$ n_t \pi_{yt} + n_{t-1} \pi_{ot} = v_t E_t $$ 

An equilibrium will be Pareto efficient if emissions taxes are chosen by conventional criteria to equate the marginal costs and discounted future benefits of emissions control:

$$ v_t = - \sum^{\bar{t}-t}_{i=1} \frac{\partial{f_{t+1}}}{\partial{T_{t+1}}} \frac{\partial{T_{t+1}}}{\partial{E_{t}}} \left(\prod^{i}_{j=1} \space \frac{1}{1+r_{t+j}} \right) $$



## Model Calibration

Noget med DICE af Nordhaus (1994) 

Starting point is 2000, where there is 5.93 billion people. Bulatao et al. (1990) states that people under the age of 35 is approximately 65% of the world's population. 

Then they parametize the number of young people alive at each date measured in billions to:

$$ N_{yt}=5.27-1.42(0.587)^t $$

Which indicate an initial population growth rate of 1.1%/year, which will converge over the long run to a population of 10.54 billion people. 

Nordhaus state gross world output $(Y_t)$, in trillion dollars per period, is a Cobb-Douglas production function:

$$ Y_t = A_t K_t^{0.25} N_t^{0.75} $$

where $A_t$ is an index of total factor productivity. 
The initial capital stock is set equal to $56 trillion.

Potential emissions, $E_{0t}$, in billion tons of carbon equivalent per period:

$$ E_{0t} = (0.181 + 0.189(0.622)^t) Y_t $$

Total factor productivity: 

$$ A_t = (235 - 142(0.739)^t) * (1-0.0133(T_t/3)^2) * (1-0.0686(1-E_t/E_{0t})^{2.89}) $$

Skrevet af dem: 
The next item to consider is the relationship between gross and net output. In Nordhaus’ specification, capital depreciates at a rate of 10%/yr. Over generational (thirty-five year) time spans, the rate of net capital investment may therefore be represented using:

$$ K_{t+1} = (1-0.1)^{35} K_t + T_t -C_t $$

DICE implies that the change in mean global temperature relative to the preindustrial norm, measured in °C, is:

$$ T_t = (5.92 log(Q_t/590) + F_t)/1.41  $$

$Q_t$ measures the effective stock of carbon dioxide and chlorofluorocarbons in the atmosphere, measured in billion tons of carbon-equivalent. The preindustrial stock is set equal to 590 units so that a doubling of greenhouse gases in the atmosphere leads to a 2.91°C temperature rise.

$F_t$ captures the so-called ‘radiative forcing’ caused by trace concentrations of methane, nitrous oxide, and water vapor as measured in watts/$m^2$. $F_t$  is exogenous such: 

$$ F_t = 1.42 - 0.764(0.523)^t $$

Finally, the atmospheric stock of carbon dioxide and chlorofluorocar-
bons is governed by the difference equation:

$$ Q_{t+1} - 590 = 0.64 E_t+(1 - 0.00833)^{35} (Q_t - 590) $$

## Numerical solution

### Model Structure

In [21]:
def utility(c_yt, c_ot1, rho):
    u_t = np.log(c_yt) + 1/(1+rho)*np.log(c_ot1)
    return u_t

In [22]:
def c_yt(k_t1, w_t, pi_yt):
    c_yt = w_t + pi_yt - k_t1
    return c_yt

In [23]:
def c_ot1(w_t1, r_t1, k_t1, pi_ot1):
    c_ot1 = w_t1 + (1+r_t1)* k_t1 + pi_ot1
    return c_ot1

In [24]:
def foc_u(r_t1, rho):
    c_ot1_c_yt = (1+r_t1)/(1+rho)
    return c_ot1_c_yt

In [ ]:
# noget med C_t + K_{t+1} = f_{t}(K_t, N_t, E_t, T_t) + K_t

In [25]:
def C_t(n, c_yt, c_ot1):
    C_t = n*c_yt + n.shift(1)*c_ot1
    return C_t

In [26]:
def K_t(n, k_t):
    K_t = n.shift(1)*k_t
    return K_t

In [27]:
def N_t(n):
    N_t = n.shift(1) + n
    return N_t

### Model Calibration

In [11]:
def N_yt(t):
    N_yt_list = []
    for i in range(t):
        N_yt = 5.27 - 1.42*0.587**i
        N_yt_list.append(N_yt)
    return N_yt_list

[3.8499999999999996, 4.436459999999999, 4.780712019999999]

In [12]:
def Y_t(A_t, K_t, N_t):
    Y_t = A_t*K_t**0.25*N_t**0.75
    return Y_t

In [13]:
def E_0t(t, Y_t):
    E_0t_list = []
    for i in range(t):
        E_0t = (0.181 + 0.189*0.622**i)*Y_t
        E_0t_list.append(E_0t)
    E_0t = E_0t_list
    return E_0t


In [14]:
def A_t(t, T_t, E_t, E_0t):
    A_t_list = []
    for i in range(t):
        A_t = (235 - 142*0.739**i)*(1-0.0133*(T_t/3)**2)*(1-0.0686*(1-E_t/E_0t)**2.89)
        A_t_list.append(A_t)
    A_t = A_t_list
    return A_t

In [15]:
def K_t1(K_t, Y_t, C_t):
    K_t1 = (1-0.1)**35*K_t + Y_t - C_t
    return K_t1

In [16]:
def T_t(Q_t, F_t):
    T_t = (5.92*np.log(Q_t/590) + F_t)/1.41
    return T_t

In [20]:
def F_t(t):
    F_t_list = []
    for i in range(t):
        F_t = 1.42 - 0.764*0.523**i
        F_t_list.append(F_t)
    F_t = F_t_list
    return F_t


[0.6559999999999999, 1.020428, 1.2110238439999998, 1.3107054704119998]

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 